# This notebook implements code to fetch noise profile from a fake backend

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator

import rustworkx as rx
from rustworkx.visualization import mpl_draw

import numpy as np

## Connect to a fake backend with AerSimulator

In [ ]:
# Need to do only once

#service = QiskitRuntimeService.save_account(
#    channel='ibm_quantum',
#    instance='ibm-q/open/main',
#    set_as_default=True,
#    overwrite=True,
#    token='<IBM Quantum API key>'
#)

In [ ]:
service = QiskitRuntimeService()

In [ ]:
dev_list = service.backends(
    filters = lambda x: x.configuration().n_qubits >= 5,
    simulator = False)

In [ ]:
print(dev_list)

In [ ]:
# Select least busy backend
dev = service.least_busy(min_num_qubits=5, simulator=False, operational=True)
print(dev)

In [ ]:
seed_sim = 42
noisy_fake_backend = AerSimulator.from_backend(dev, seed_simulator=seed_sim)
print(noisy_fake_backend)

## Construct hardware topology graph of the backend

In [ ]:
graph = rx.PyDiGraph()

In [ ]:
noisy_fake_backend.num_qubits

In [ ]:
graph.add_nodes_from(np.arange(0, noisy_fake_backend.num_qubits, 1))

In [ ]:
two_qubit_gate='ecr'

In [ ]:
noisy_fake_backend.coupling_map.connected_components()

In [ ]:
graph.add_edges_from(
    [
        (
            edge[0],
            edge[1],
            noisy_fake_backend.properties().gate_error(gate=two_qubit_gate, qubits=(edge[0], edge[1]))
        )
        for edge in noisy_fake_backend.coupling_map
    ]
)


In [ ]:
mpl_draw(graph, with_labels=True)

## Noise profile of each qubit

In [ ]:
from qiskit.visualization import plot_error_map

In [ ]:
plot_error_map(noisy_fake_backend, figsize=(30, 24))

In [ ]:
Noise_dict = dict()

In [ ]:
backend_name = noisy_fake_backend.name
num_qubits = noisy_fake_backend.num_qubits
cmap = noisy_fake_backend.coupling_map
two_q_error_map = {}
single_gate_errors = [0] * num_qubits
read_err = [0] * num_qubits
cx_errors = []
for gate, prop_dict in noisy_fake_backend.target.items():
    if prop_dict is None or None in prop_dict:
        continue
    for qargs, inst_props in prop_dict.items():
        if inst_props is None:
            continue
        if gate == "measure":
            if inst_props.error is not None:
                read_err[qargs[0]] = inst_props.error
                Noise_dict[f"{qargs[0]}"] = inst_props.error
        elif len(qargs) == 1:
            if inst_props.error is not None:
                single_gate_errors[qargs[0]] = max(
                    single_gate_errors[qargs[0]], inst_props.error
                )
        elif len(qargs) == 2:
            if inst_props.error is not None:
                two_q_error_map[qargs] = max(two_q_error_map.get(qargs, 0), inst_props.error)

In [ ]:
len(read_err)

In [ ]:
Noise_dict